## InceptionV3 Model:

---
Developed by: Christian Szegedy, Wei Liu, Yangqing Jia, Pierre Sermanet, Scott Reed, Dragomir Anguelov, Dumitru Erhan, Vincent Vanhoucke, and Andrew Rabinovich

Published in: 2015

Primary Use Case: Image classification

---
### Architecture:
Input Layer: 299x299 RGB images

Initial Convolution and Pooling:

3x3 convolution with 32 filters, stride 2

3x3 convolution with 32 filters

3x3 convolution with 64 filters, padding 1

3x3 max pooling with stride 2

1x1 convolution with 80 filters

3x3 convolution with 192 filters

3x3 max pooling with stride 2

Inception Modules:

1st Inception Module: Combination of 1x1, 3x3, 5x5 convolutions, and 3x3 max pooling

2nd Inception Module: Same structure as the 1st Inception Module with different filter sizes

Global Average Pooling: Converts feature maps into a 1D feature vector

Fully Connected Layer: Number of classes (usually 1000 for ImageNet)

Dropout: Applied before the fully connected layer to prevent overfitting

Activation Functions: Uses ReLU activation function after each convolutional layer

---
### Key Features:
Inception Modules: Combines multiple convolutional filters of different sizes to capture various features at different scales.

Factorized Convolutions: Efficient computation by factorizing convolutions into smaller operations (e.g., 3x3 into two 1x3 and 3x1 convolutions).

High Performance: Achieved state-of-the-art results on the ImageNet dataset at the time of its release.

---
### Applications:
Image Classification: Used for classifying images into different categories.

Feature Extraction: Pre-trained InceptionV3 models are often used to extract features from images for transfer learning in other tasks.

---
### Impact:
Influence: InceptionV3 has influenced the design of many subsequent deep learning models with its innovative inception modules.

Benchmark: Often used as a benchmark for comparing the performance of new models and techniques in the field of computer vision.

---

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class InceptionModule(nn.Module):
    def __init__(self, in_channels, out1x1, out3x3red, out3x3, out5x5red, out5x5, out_pool):
        super(InceptionModule, self).__init__()
        
        self.branch1 = nn.Conv2d(in_channels, out1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, out3x3red, kernel_size=1),
            nn.Conv2d(out3x3red, out3x3, kernel_size=3, padding=1)
        )

        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, out5x5red, kernel_size=1),
            nn.Conv2d(out5x5red, out5x5, kernel_size=5, padding=2)
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, out_pool, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        outputs = [branch1, branch2, branch3, branch4]
        return torch.cat(outputs, 1)

class InceptionV3(nn.Module):
    def __init__(self, num_classes=1000):
        super(InceptionV3, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2)

        self.conv4 = nn.Conv2d(64, 80, kernel_size=1)
        self.conv5 = nn.Conv2d(80, 192, kernel_size=3)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2)

        self.inception1 = InceptionModule(192, 64, 48, 64, 64, 96, 32)
        self.inception2 = InceptionModule(256, 64, 48, 64, 64, 96, 64)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(288, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.maxpool1(x)

        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.maxpool2(x)

        x = self.inception1(x)
        x = self.inception2(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x



In [5]:
# Create an instance of the InceptionV3 model
model = InceptionV3(num_classes=1000)

In [6]:
model

InceptionV3(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1))
  (conv5): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (inception1): InceptionModule(
    (branch1): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
    (branch2): Sequential(
      (0): Conv2d(192, 48, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (branch3): Sequential(
      (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(64, 96, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    )
    (branch4): Sequential(
      (0): MaxPool